In [166]:
import pandas as pd
import numpy as np
import random
import os
import gc
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [167]:
# def csv_to_parquet(csv_path, save_name):
#     df = pd.read_csv(csv_path)
#     df.to_parquet(f'./{save_name}.parquet')
#     del df
#     gc.collect()
#     print(save_name, 'Done.')
    
# csv_to_parquet('./train.csv', 'train')
# csv_to_parquet('./test.csv', 'test')

In [168]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [169]:
train = train.drop('Carrier_Code(IATA)',axis=1)

In [170]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 18 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport            1000000 non-null  object 
 8   Origin_Airport_ID         1000000 non-null  int64  
 9   Origin_State              890985 non-null   object 
 10  Destination_Airport       1000000 non-null  object 
 11  Destination_Airport_ID    1000000 non-null  int64  
 12  Destination_State         890921 non-null   object 
 13  Distance                  10

In [171]:
print(len(train['Origin_Airport'].unique()) == len(train['Origin_Airport_ID'].unique()))
train = train.drop('Origin_Airport',axis=1)
print(len(train['Destination_Airport'].unique()) == len(train['Destination_Airport_ID'].unique()))
train = train.drop('Destination_Airport',axis=1)
train

True
True


,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Origin_State,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,NaN,NaN,0,0,13851,Oklahoma,12191,Texas,419.0,Southwest Airlines Co.,19393.0,N7858A,None
1,TRAIN_000001,8,15,740.0,1024.0,0,0,13930,Illinois,14869,Utah,1250.0,SkyWest Airlines Inc.,20304.0,N125SY,None
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,11057,North Carolina,12953,New York,544.0,American Airlines Inc.,19805.0,N103US,None
3,TRAIN_000003,7,10,905.0,1735.0,0,0,12892,California,11618,New Jersey,2454.0,United Air Lines Inc.,NaN,N595UA,None
4,TRAIN_000004,1,11,900.0,1019.0,0,0,14771,California,10157,California,250.0,SkyWest Airlines Inc.,20304.0,N161SY,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,13930,None,14100,None,678.0,United Air Lines Inc.,19977.0,N477UA,None
999996,TRAIN_999996,5,30,920.0,1028.0,0,0,11637,None,13487,Minnesota,223.0,SkyWest Airlines Inc.,NaN,N439SW,None
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,13796,None,12191,Texas,1642.0,Southwest Airlines Co.,19393.0,N230WN,None
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,10693,Tennessee,10397,None,214.0,Delta Air Lines Inc.,19790.0,N968DL,None


In [172]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 16 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport_ID         1000000 non-null  int64  
 8   Origin_State              890985 non-null   object 
 9   Destination_Airport_ID    1000000 non-null  int64  
 10  Destination_State         890921 non-null   object 
 11  Distance                  1000000 non-null  float64
 12  Airline                   891080 non-null   object 
 13  Carrier_ID(DOT)           89

In [173]:
tar = train['Origin_State'].unique().tolist()
tar = [str(i) for i in tar]
tar.sort()
tar1 = train['Destination_State'].unique().tolist()
tar1 = [str(i) for i in tar]
tar1.sort()
print(tar)
print(tar1)

train = train.drop(['Origin_State','Destination_State'],axis=1)

['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'None', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'U.S. Pacific Trust Territories and Possessions', 'U.S. Virgin Islands', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',

In [174]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 14 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport_ID         1000000 non-null  int64  
 8   Destination_Airport_ID    1000000 non-null  int64  
 9   Distance                  1000000 non-null  float64
 10  Airline                   891080 non-null   object 
 11  Carrier_ID(DOT)           891003 non-null   float64
 12  Tail_Number               1000000 non-null  object 
 13  Delay                     25

In [175]:
print(train['Airline'].value_counts())
print(train['Carrier_ID(DOT)'].value_counts())
train = train.drop(['Airline'],axis=1)

Southwest Airlines Co.                       182113
Delta Air Lines Inc.                          89750
SkyWest Airlines Inc.                         86867
United Air Lines Inc.                         84070
American Airlines Inc.                        82755
JetBlue Airways                               41643
Republic Airlines                             33582
Alaska Airlines Inc.                          29141
Envoy Air                                     27455
Spirit Air Lines                              24998
Comair Inc.                                   24723
Mesa Airlines Inc.                            24679
Endeavor Air Inc.                             23172
ExpressJet Airlines Inc.                      20144
Frontier Airlines Inc.                        17415
Allegiant Air                                 13667
Air Wisconsin Airlines Corp                   13400
Horizon Air                                   13087
Hawaiian Airlines Inc.                        10907
Capital Carg

In [176]:
lbe = LabelEncoder()
train['Tail_Number'] = lbe.fit_transform(train['Tail_Number'])
train = train.drop('ID',axis=1)

In [177]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Month                     1000000 non-null  int64  
 1   Day_of_Month              1000000 non-null  int64  
 2   Estimated_Departure_Time  890981 non-null   float64
 3   Estimated_Arrival_Time    890960 non-null   float64
 4   Cancelled                 1000000 non-null  int64  
 5   Diverted                  1000000 non-null  int64  
 6   Origin_Airport_ID         1000000 non-null  int64  
 7   Destination_Airport_ID    1000000 non-null  int64  
 8   Distance                  1000000 non-null  float64
 9   Carrier_ID(DOT)           891003 non-null   float64
 10  Tail_Number               1000000 non-null  int64  
 11  Delay                     255001 non-null   object 
dtypes: float64(4), int64(7), object(1)
memory usage: 91.6+ MB


In [183]:
test_ = train[train['Delay'].isnull()]
# test_ = test_.dropna()
test_ = test_.drop('Delay',axis=1)
test_ = test_.dropna()
test_

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number
1,8,15,740.0,1024.0,0,0,13930,14869,1250.0,20304.0,310
2,9,6,1610.0,1805.0,0,0,11057,12953,544.0,19805.0,140
4,1,11,900.0,1019.0,0,0,14771,10157,250.0,20304.0,556
7,4,20,1815.0,1955.0,0,0,13930,13198,403.0,20304.0,173
9,6,6,650.0,838.0,0,0,12992,12266,374.0,20366.0,468
...,...,...,...,...,...,...,...,...,...,...,...
999993,10,17,603.0,755.0,0,0,12264,11292,1452.0,20436.0,1538
999995,9,18,936.0,1243.0,0,0,13930,14100,678.0,19977.0,2477
999997,6,28,800.0,1340.0,0,0,13796,12191,1642.0,19393.0,994
999998,9,27,1613.0,1824.0,0,0,10693,10397,214.0,19790.0,6207


In [179]:
train_ = pd.DataFrame()
train_ = pd.concat([train_,train[train['Delay']=='Not_Delayed']])
train_ = pd.concat([train_,train[train['Delay']=='Delayed']])
train_ = train_.dropna()
train_

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
8,6,13,1420.0,1550.0,0,0,10821,11057,361.0,19393.0,4083,Not_Delayed
13,9,19,615.0,706.0,0,0,13158,11292,563.0,20304.0,579,Not_Delayed
19,7,14,1907.0,2145.0,0,0,10529,10397,859.0,19790.0,6147,Not_Delayed
32,6,20,750.0,1014.0,0,0,13930,12945,323.0,20366.0,327,Not_Delayed
42,10,24,1005.0,1130.0,0,0,13930,15016,258.0,20237.0,4766,Not_Delayed
...,...,...,...,...,...,...,...,...,...,...,...,...
999816,11,6,2115.0,2348.0,0,0,13930,11076,360.0,20304.0,6171,Delayed
999843,6,13,1749.0,1850.0,0,0,13342,13930,67.0,20237.0,263,Delayed
999955,4,12,1310.0,1356.0,0,0,14747,14057,129.0,19687.0,2250,Delayed
999963,5,2,1759.0,1926.0,0,0,12953,11278,214.0,20452.0,5204,Delayed


In [185]:
target = 'Delay'
features = ['Month',
 'Day_of_Month',
 'Estimated_Departure_Time',
 'Estimated_Arrival_Time',
 'Cancelled',
 'Diverted',
 'Origin_Airport_ID',
 'Destination_Airport_ID',
 'Distance',
 'Carrier_ID(DOT)',
 'Tail_Number']

In [190]:
train_ = train_.reset_index(drop=True)

In [191]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

kf = KFold(n_splits = 10, random_state = 41, shuffle = True)
models = []
val_scores = []
preds = []

params = {
'n_estimators':5000,
'max_depth':6,
'early_stopping_rounds':200,
'learning_rate':0.01,
'one_hot_max_size':3,
'bootstrap_type':"MVS",
'l2_leaf_reg':2,
'random_state':41
         }

# bundle = [CatBoostClassifier, LGBMClassifier, XGBClassifier]
# for model in bundle:
# model = CatBoostClassifier(**params)
for i, (train_index, val_index) in tqdm(enumerate(kf.split(train_, train_[target]))):

        # clf.fit(train_x, train_y)

        X_train, X_val = train_[features].loc[train_index], train_[features].loc[val_index]
        y_train, y_val = train_[target][train_index], train_[target][val_index]

        model = CatBoostClassifier(**params)

        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 2000)

        models.append(model)
        score = log_loss(y_val, model.predict_proba(X_val))

        val_scores.append(score)

0it [00:00, ?it/s]

0:	learn: 0.6878246	test: 0.6878318	best: 0.6878318 (0)	total: 73.7ms	remaining: 6m 8s
2000:	learn: 0.4346923	test: 0.4399201	best: 0.4399201 (2000)	total: 19.7s	remaining: 29.5s
4000:	learn: 0.4256457	test: 0.4381414	best: 0.4381414 (4000)	total: 40.9s	remaining: 10.2s


1it [00:51, 51.94s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4379769934
bestIteration = 4781

Shrink model to first 4782 iterations.
0:	learn: 0.6878312	test: 0.6877737	best: 0.6877737 (0)	total: 11.5ms	remaining: 57.6s
2000:	learn: 0.4349094	test: 0.4379959	best: 0.4379959 (2000)	total: 20.8s	remaining: 31.1s
4000:	learn: 0.4258807	test: 0.4362211	best: 0.4362172 (3991)	total: 42s	remaining: 10.5s


2it [01:45, 52.67s/it]

...

4999:	learn: 0.4218350	test: 0.4396035	best: 0.4396015 (4976)	total: 52.5s	remaining: 0us

bestTest = 0.4396014641
bestIteration = 4976

Shrink model to first 4977 iterations.
0:	learn: 0.6878323	test: 0.6877875	best: 0.6877875 (0)	total: 10.2ms	remaining: 51s
2000:	learn: 0.4349536	test: 0.4371951	best: 0.4371951 (2000)	total: 20.7s	remaining: 31.1s
4000:	learn: 0.4260268	test: 0.4359337	best: 0.4358961 (3914)	total: 41.9s	remaining: 10.5s


9it [07:34, 48.92s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4358960655
bestIteration = 3914

Shrink model to first 3915 iterations.
0:	learn: 0.6878307	test: 0.6877816	best: 0.6877816 (0)	total: 11.4ms	remaining: 57.2s
2000:	learn: 0.4350448	test: 0.4360157	best: 0.4360157 (2000)	total: 20.8s	remaining: 31.1s
4000:	learn: 0.4261070	test: 0.4345308	best: 0.4345214 (3988)	total: 42s	remaining: 10.5s


10it [08:25, 50.55s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4343794712
bestIteration = 4627

Shrink model to first 4628 iterations.


In [192]:
val_scores

[0.43797699335876283,
 0.4358958149772492,
 0.43636692508490377,
 0.4353263327329921,
 0.4463164023101533,
 0.4463811067884916,
 0.4411287296930336,
 0.439601464129778,
 0.4358960654834986,
 0.43437947119816384]

In [193]:
cat_preds = []
for model in models[:10]:
    preds = model.predict_proba(test_[features])
    cat_preds.append(preds)

In [198]:
pred = np.array(cat_preds).mean(axis=0)
test_['Delay'] = [np.argmax(i) for i in pred]
test_

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
1,8,15,740.0,1024.0,0,0,13930,14869,1250.0,20304.0,310,1
2,9,6,1610.0,1805.0,0,0,11057,12953,544.0,19805.0,140,1
4,1,11,900.0,1019.0,0,0,14771,10157,250.0,20304.0,556,1
7,4,20,1815.0,1955.0,0,0,13930,13198,403.0,20304.0,173,1
9,6,6,650.0,838.0,0,0,12992,12266,374.0,20366.0,468,1
...,...,...,...,...,...,...,...,...,...,...,...,...
999993,10,17,603.0,755.0,0,0,12264,11292,1452.0,20436.0,1538,1
999995,9,18,936.0,1243.0,0,0,13930,14100,678.0,19977.0,2477,1
999997,6,28,800.0,1340.0,0,0,13796,12191,1642.0,19393.0,994,1
999998,9,27,1613.0,1824.0,0,0,10693,10397,214.0,19790.0,6207,1


In [203]:
model.classes_

array(['Delayed', 'Not_Delayed'], dtype=object)

In [230]:
lbe = LabelEncoder()
train_['Delay'] = lbe.fit_transform(train_['Delay'])
lbe.classes_

array(['Delayed', 'Not_Delayed'], dtype=object)

In [233]:
train_

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
0,6,13,1420.0,1550.0,0,0,10821,11057,361.0,19393.0,4083,1
1,9,19,615.0,706.0,0,0,13158,11292,563.0,20304.0,579,1
2,7,14,1907.0,2145.0,0,0,10529,10397,859.0,19790.0,6147,1
3,6,20,750.0,1014.0,0,0,13930,12945,323.0,20366.0,327,1
4,10,24,1005.0,1130.0,0,0,13930,15016,258.0,20237.0,4766,1
...,...,...,...,...,...,...,...,...,...,...,...,...
180470,11,6,2115.0,2348.0,0,0,13930,11076,360.0,20304.0,6171,0
180471,6,13,1749.0,1850.0,0,0,13342,13930,67.0,20237.0,263,0
180472,4,12,1310.0,1356.0,0,0,14747,14057,129.0,19687.0,2250,0
180473,5,2,1759.0,1926.0,0,0,12953,11278,214.0,20452.0,5204,0


In [234]:
test_

,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
1,8,15,740.0,1024.0,0,0,13930,14869,1250.0,20304.0,310,1
2,9,6,1610.0,1805.0,0,0,11057,12953,544.0,19805.0,140,1
4,1,11,900.0,1019.0,0,0,14771,10157,250.0,20304.0,556,1
7,4,20,1815.0,1955.0,0,0,13930,13198,403.0,20304.0,173,1
9,6,6,650.0,838.0,0,0,12992,12266,374.0,20366.0,468,1
...,...,...,...,...,...,...,...,...,...,...,...,...
999993,10,17,603.0,755.0,0,0,12264,11292,1452.0,20436.0,1538,1
999995,9,18,936.0,1243.0,0,0,13930,14100,678.0,19977.0,2477,1
999997,6,28,800.0,1340.0,0,0,13796,12191,1642.0,19393.0,994,1
999998,9,27,1613.0,1824.0,0,0,10693,10397,214.0,19790.0,6207,1


In [12]:
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
#     mode = train[col].mode()[0]
#     train[col] = train[col].fillna(mode)
    
    if col in test.columns:
        test[col] = test[col].fillna(train[col].mode()[0])
print('Done.')

qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

Done.
Done.


In [13]:
# train = train.dropna()
# column_number = {}
# for i, column in enumerate(sample_submission.columns):
#     column_number[column] = i
    
# def to_number(x, dic):
#     return dic[x]

# train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
# print('Done.')

In [14]:
# train['Delay'] = le.fit_transform(train['Delay'])

In [15]:
train = train.drop(columns=['ID'])
train = train.reset_index(drop=True)

# train_y = train['Delay_num']
# train_y = train_y.reset_index(drop=True)

test = test.drop(columns=['ID'])

In [17]:
target = 'Delay'
features = [c for c in train.columns if c not in [target]]

In [19]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

kf = KFold(n_splits = 10, random_state = 41, shuffle = True)
models = []
val_scores = []
preds = []

params = {
'n_estimators':5000,
'max_depth':6,
'early_stopping_rounds':200,
'learning_rate':0.01,
'one_hot_max_size':3,
'bootstrap_type':"MVS",
'l2_leaf_reg':2,
'random_state':41
         }

# bundle = [CatBoostClassifier, LGBMClassifier, XGBClassifier]
# for model in bundle:
# model = CatBoostClassifier(**params)
for i, (train_index, val_index) in tqdm(enumerate(kf.split(train, train[target]))):

        # clf.fit(train_x, train_y)

        X_train, X_val = train[features].loc[train_index], train[features].loc[val_index]
        y_train, y_val = train[target][train_index], train[target][val_index]

        model = CatBoostClassifier(**params)

        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 2000)

        models.append(model)
        score = log_loss(y_val, model.predict_proba(X_val))

        val_scores.append(score)

0it [00:00, ?it/s]

0:	learn: 0.6878222	test: 0.6877996	best: 0.6877996 (0)	total: 65.5ms	remaining: 5m 27s
2000:	learn: 0.4282721	test: 0.4355061	best: 0.4355061 (2000)	total: 17.3s	remaining: 25.9s
4000:	learn: 0.4145342	test: 0.4346265	best: 0.4345817 (3881)	total: 35.3s	remaining: 8.8s


1it [00:36, 36.49s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4345817178
bestIteration = 3881

Shrink model to first 3882 iterations.
0:	learn: 0.6878308	test: 0.6877573	best: 0.6877573 (0)	total: 9.58ms	remaining: 47.9s
2000:	learn: 0.4281822	test: 0.4368638	best: 0.4368638 (2000)	total: 17.5s	remaining: 26.2s
4000:	learn: 0.4145818	test: 0.4357239	best: 0.4357239 (4000)	total: 35.1s	remaining: 8.77s


2it [01:16, 38.45s/it]

...

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4364941164
bestIteration = 4382

Shrink model to first 4383 iterations.
0:	learn: 0.6877933	test: 0.6878703	best: 0.6878703 (0)	total: 11.4ms	remaining: 56.9s
2000:	learn: 0.4270324	test: 0.4471126	best: 0.4471126 (2000)	total: 17.9s	remaining: 26.9s


9it [05:48, 39.01s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4462912209
bestIteration = 3682

Shrink model to first 3683 iterations.
0:	learn: 0.6878013	test: 0.6878305	best: 0.6878305 (0)	total: 9.21ms	remaining: 46s
2000:	learn: 0.4276933	test: 0.4419723	best: 0.4419723 (2000)	total: 19s	remaining: 28.4s
4000:	learn: 0.4141680	test: 0.4412495	best: 0.4412383 (3856)	total: 37.4s	remaining: 9.34s


10it [06:26, 38.66s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4412382792
bestIteration = 3856

Shrink model to first 3857 iterations.


In [20]:
val_scores

[0.43458171775239673,
 0.4356648996813724,
 0.4383582894949309,
 0.44565824682683775,
 0.43242458927469923,
 0.4353388641482646,
 0.43785904076377596,
 0.43649411638636254,
 0.44629122094531626,
 0.4412382792435065]

In [21]:
cat_preds = []
for model in models[:10]:
    preds = model.predict_proba(test[features])
    cat_preds.append(preds)

In [33]:
cat_preds

[array([[0.22113619, 0.77886381],
        [0.19566839, 0.80433161],
        [0.31759269, 0.68240731],
        ...,
        [0.25149195, 0.74850805],
        [0.13303926, 0.86696074],
        [0.12975944, 0.87024056]]),
 array([[0.16978971, 0.83021029],
        [0.19426276, 0.80573724],
        [0.30265477, 0.69734523],
        ...,
        [0.25165676, 0.74834324],
        [0.11926982, 0.88073018],
        [0.12793548, 0.87206452]]),
 array([[0.18750157, 0.81249843],
        [0.20600966, 0.79399034],
        [0.31656755, 0.68343245],
        ...,
        [0.26617674, 0.73382326],
        [0.11567346, 0.88432654],
        [0.13150554, 0.86849446]]),
 array([[0.18531055, 0.81468945],
        [0.180164  , 0.819836  ],
        [0.3176459 , 0.6823541 ],
        ...,
        [0.26510609, 0.73489391],
        [0.12152297, 0.87847703],
        [0.12414252, 0.87585748]]),
 array([[0.18999597, 0.81000403],
        [0.19659713, 0.80340287],
        [0.35342903, 0.64657097],
        ...,
        [

In [22]:
pred = np.array(cat_preds).mean(axis=0)

submission = pd.DataFrame(data=pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('baseline_submission.csv', index=True)